# Setup

In [5]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import layers
from diffusion_model_for_speech import *
from wavenet import *
from speech_dataset import *
import math
import numpy as np
import glob

# Hyperparameters

In [6]:
diffusion_steps = 20
learning_rate = 2e-4
weight_decay = 1e-5
checkpoint_path = "checkpoints/diffusion_model_for_voice/diffusion_steps_%d"%(diffusion_steps)
final_checkpoint_path = "checkpoints/diffusion_model_for_voice/final_diffusion_steps_%d"%(diffusion_steps)

# Data Pipeline

In [7]:
dataset = Speech_dataset()

dataset size: 180
sample size: (66270,)


2023-04-01 14:38:47.496899: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22292 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6
2023-04-01 14:38:48.005466: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [4]:
dataset_tf = dataset.audio_dataset()

In [5]:
for signal, logmel in dataset_tf.take(5):
    print("Signal shape:", signal.shape)
    print("Log Mel shape:", logmel.shape)

Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)


# Training

In [6]:
model = DiffusionModel(diffusion_steps = diffusion_steps)

model.compile(
    optimizer=tfa.optimizers.AdamW(
        learning_rate=learning_rate, 
        weight_decay=weight_decay),
    loss=keras.losses.MeanSquaredError(),  # pixelwise mean absolute error is used as loss, mse also ok
    #run_eagerly=True
)

# save the best model based on the validation KID metric
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="mel_loss",
    mode="min",
    save_best_only=True,
)

final_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=final_checkpoint_path,
    save_weights_only=True,
)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='noise_loss',  
    factor=0.5,  
    patience=5,  
    verbose=1,  
    min_lr=1e-6  
)
# calculate mean and variance of training dataset for normalization
#model.normalizer.adapt(dataset_tf)
#model.load_weights(checkpoint_path)

total_size = [i for i,_ in enumerate(dataset_tf)][-1] + 1

val_split = 0.1
val_size = int(total_size * val_split)
train_size = total_size - val_size

dataset_tf = dataset_tf.shuffle(buffer_size=total_size)
train_dataset = dataset_tf.take(train_size)
val_dataset = dataset_tf.skip(train_size)


In [7]:
for signal, logmel in train_dataset.take(5):
    print("Signal shape:", signal.shape)
    print("Log Mel shape:", logmel.shape)

Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)
Signal shape: (8, 6400)
Log Mel shape: (8, 25, 80)


In [ ]:
#if put tensor into fit directly, it can't use the validation_data. please use the validation_split to split the data into training and validation.
model.fit(
    dataset_tf,
    epochs=num_epochs,
    validation_data=val_dataset,
    callbacks=[
        #keras.callbacks.LambdaCallback(on_epoch_end=model.to_speech), 
        checkpoint_callback,
        final_checkpoint_callback,
        #reduce_lr_callback 
    ],
)

Epoch 1/10000


2023-04-01 12:05:41.559250: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2023-04-01 12:05:44.252171: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


23/23 [==============================] - 24s 267ms/step - noise_loss: 0.7940 - signal_loss: 0.1991 - mel_loss: 38.4263 - val_noise_loss: 0.9655 - val_signal_loss: 0.2599 - val_mel_loss: 41.1750
Epoch 2/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.7654 - signal_loss: 0.1778 - mel_loss: 38.9785 - val_noise_loss: 0.8416 - val_signal_loss: 0.1551 - val_mel_loss: 36.6523
Epoch 3/10000
23/23 [==============================] - 4s 176ms/step - noise_loss: 0.6218 - signal_loss: 0.1079 - mel_loss: 38.8295 - val_noise_loss: 0.2935 - val_signal_loss: 0.0319 - val_mel_loss: 32.1691
Epoch 4/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.4543 - signal_loss: 0.0372 - mel_loss: 30.4609 - val_noise_loss: 0.2734 - val_signal_loss: 0.0205 - val_mel_loss: 36.7000
Epoch 5/10000
23/23 [==============================] - 4s 174ms/step - noise_loss: 0.3105 - signal_loss: 0.0165 - mel_loss: 31.2893 - val_noise_loss: 0.0908 - val_signal_loss: 0.0113 - 

23/23 [==============================] - 4s 174ms/step - noise_loss: 0.0931 - signal_loss: 7.0191e-04 - mel_loss: 15.3403 - val_noise_loss: 0.1661 - val_signal_loss: 4.8050e-04 - val_mel_loss: 17.3440
Epoch 79/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.0736 - signal_loss: 5.8592e-04 - mel_loss: 14.3773 - val_noise_loss: 0.0034 - val_signal_loss: 4.0272e-04 - val_mel_loss: 15.8971
Epoch 80/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.0838 - signal_loss: 5.8673e-04 - mel_loss: 14.0588 - val_noise_loss: 0.1786 - val_signal_loss: 5.3968e-04 - val_mel_loss: 15.5405
Epoch 81/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.1077 - signal_loss: 7.1538e-04 - mel_loss: 12.1023 - val_noise_loss: 0.0105 - val_signal_loss: 4.1038e-04 - val_mel_loss: 15.6568
Epoch 82/10000
23/23 [==============================] - 4s 176ms/step - noise_loss: 0.0871 - signal_loss: 6.6161e-04 - mel_loss: 12.1482 - val_noise_lo

23/23 [==============================] - 4s 171ms/step - noise_loss: 0.0908 - signal_loss: 7.1926e-04 - mel_loss: 13.2703 - val_noise_loss: 0.2238 - val_signal_loss: 4.7138e-04 - val_mel_loss: 18.3013
Epoch 155/10000
23/23 [==============================] - 4s 174ms/step - noise_loss: 0.1043 - signal_loss: 5.5562e-04 - mel_loss: 13.9314 - val_noise_loss: 0.0229 - val_signal_loss: 0.0011 - val_mel_loss: 15.6843
Epoch 156/10000
23/23 [==============================] - 4s 172ms/step - noise_loss: 0.0894 - signal_loss: 7.3576e-04 - mel_loss: 14.5500 - val_noise_loss: 0.0057 - val_signal_loss: 9.5500e-04 - val_mel_loss: 16.6578
Epoch 157/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.0885 - signal_loss: 5.9221e-04 - mel_loss: 12.7268 - val_noise_loss: 0.1463 - val_signal_loss: 6.2964e-04 - val_mel_loss: 16.6726
Epoch 158/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.1087 - signal_loss: 8.5417e-04 - mel_loss: 12.0264 - val_noise_lo

...

Epoch 344/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.0686 - signal_loss: 5.0118e-04 - mel_loss: 11.5516 - val_noise_loss: 0.0568 - val_signal_loss: 2.0194e-04 - val_mel_loss: 11.9169
Epoch 345/10000
23/23 [==============================] - 4s 172ms/step - noise_loss: 0.0746 - signal_loss: 5.5689e-04 - mel_loss: 12.0258 - val_noise_loss: 0.0185 - val_signal_loss: 0.0019 - val_mel_loss: 14.7280
Epoch 346/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.0824 - signal_loss: 5.8793e-04 - mel_loss: 12.6008 - val_noise_loss: 0.0560 - val_signal_loss: 2.5487e-04 - val_mel_loss: 19.2055
Epoch 347/10000
23/23 [==============================] - 4s 172ms/step - noise_loss: 0.0893 - signal_loss: 5.6532e-04 - mel_loss: 12.3789 - val_noise_loss: 0.1116 - val_signal_loss: 4.5430e-04 - val_mel_loss: 7.4733
Epoch 348/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.0690 - signal_loss: 6.2514e-04 - mel_loss: 10.8248

Epoch 382/10000
23/23 [==============================] - 4s 176ms/step - noise_loss: 0.0854 - signal_loss: 5.4864e-04 - mel_loss: 12.2156 - val_noise_loss: 0.0199 - val_signal_loss: 7.3901e-04 - val_mel_loss: 14.5573
Epoch 383/10000
23/23 [==============================] - 4s 171ms/step - noise_loss: 0.0772 - signal_loss: 6.4531e-04 - mel_loss: 9.4980 - val_noise_loss: 0.0107 - val_signal_loss: 5.0697e-04 - val_mel_loss: 6.4618
Epoch 384/10000
23/23 [==============================] - 4s 172ms/step - noise_loss: 0.0850 - signal_loss: 7.3388e-04 - mel_loss: 10.9796 - val_noise_loss: 0.0679 - val_signal_loss: 4.2448e-04 - val_mel_loss: 7.0245
Epoch 385/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.0841 - signal_loss: 5.5197e-04 - mel_loss: 9.5372 - val_noise_loss: 0.0532 - val_signal_loss: 4.2310e-04 - val_mel_loss: 9.4133
Epoch 386/10000
23/23 [==============================] - 4s 174ms/step - noise_loss: 0.1011 - signal_loss: 5.9694e-04 - mel_loss: 12.2533

Epoch 420/10000
23/23 [==============================] - 4s 172ms/step - noise_loss: 0.0684 - signal_loss: 5.6705e-04 - mel_loss: 10.0826 - val_noise_loss: 0.1650 - val_signal_loss: 2.3492e-04 - val_mel_loss: 14.3674
Epoch 421/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.0736 - signal_loss: 5.8377e-04 - mel_loss: 9.8139 - val_noise_loss: 0.0485 - val_signal_loss: 3.1479e-04 - val_mel_loss: 13.8810
Epoch 422/10000
23/23 [==============================] - 4s 177ms/step - noise_loss: 0.0791 - signal_loss: 6.8220e-04 - mel_loss: 11.3672 - val_noise_loss: 0.0024 - val_signal_loss: 3.6064e-04 - val_mel_loss: 12.1428
Epoch 423/10000
23/23 [==============================] - 4s 176ms/step - noise_loss: 0.0941 - signal_loss: 5.0551e-04 - mel_loss: 9.6280 - val_noise_loss: 0.0053 - val_signal_loss: 7.2605e-04 - val_mel_loss: 8.1093
Epoch 424/10000
23/23 [==============================] - 4s 174ms/step - noise_loss: 0.0864 - signal_loss: 5.8273e-04 - mel_loss: 11.21

Epoch 458/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.0637 - signal_loss: 4.3106e-04 - mel_loss: 11.0788 - val_noise_loss: 0.0037 - val_signal_loss: 2.7088e-04 - val_mel_loss: 13.3126
Epoch 459/10000
23/23 [==============================] - 4s 171ms/step - noise_loss: 0.0646 - signal_loss: 3.4652e-04 - mel_loss: 11.4224 - val_noise_loss: 0.0037 - val_signal_loss: 4.9011e-04 - val_mel_loss: 14.8703
Epoch 460/10000
23/23 [==============================] - 4s 176ms/step - noise_loss: 0.0758 - signal_loss: 7.4506e-04 - mel_loss: 11.4356 - val_noise_loss: 0.0725 - val_signal_loss: 6.0012e-04 - val_mel_loss: 4.5776
Epoch 461/10000
23/23 [==============================] - 4s 174ms/step - noise_loss: 0.0678 - signal_loss: 5.1173e-04 - mel_loss: 11.2638 - val_noise_loss: 0.0494 - val_signal_loss: 0.0011 - val_mel_loss: 3.3742
Epoch 462/10000
23/23 [==============================] - 4s 175ms/step - noise_loss: 0.0664 - signal_loss: 5.1401e-04 - mel_loss: 11.9110 

Epoch 496/10000
23/23 [==============================] - 4s 171ms/step - noise_loss: 0.0578 - signal_loss: 4.9388e-04 - mel_loss: 10.8951 - val_noise_loss: 0.0096 - val_signal_loss: 2.3148e-04 - val_mel_loss: 18.5070
Epoch 497/10000
23/23 [==============================] - 4s 178ms/step - noise_loss: 0.0631 - signal_loss: 4.8058e-04 - mel_loss: 10.9345 - val_noise_loss: 0.0200 - val_signal_loss: 4.4442e-04 - val_mel_loss: 10.1640
Epoch 498/10000
23/23 [==============================] - 4s 176ms/step - noise_loss: 0.0617 - signal_loss: 4.7236e-04 - mel_loss: 10.8158 - val_noise_loss: 0.0135 - val_signal_loss: 5.7156e-04 - val_mel_loss: 6.2909
Epoch 499/10000
23/23 [==============================] - 4s 174ms/step - noise_loss: 0.0566 - signal_loss: 3.9138e-04 - mel_loss: 10.1734 - val_noise_loss: 0.0328 - val_signal_loss: 2.2435e-04 - val_mel_loss: 13.0363
Epoch 500/10000
23/23 [==============================] - 4s 173ms/step - noise_loss: 0.0693 - signal_loss: 6.1930e-04 - mel_loss: 7.0

# inference

In [8]:
speech = next(iter(dataset.rawset))
speech = speech[:speech.shape[0] // 256 * 256]
librosa.output.write_wav(os.path.join('/home/lqm/Diffusion-Bison/out/',  'test_gt.wav'),speech.numpy(),16000)
noise = tf.random.normal(tf.shape(speech[None]))
librosa.output.write_wav(
        os.path.join('/home/lqm/Diffusion-Bison/out/' , 'test_noise.wav'),noise[0].numpy(),16000)

In [9]:
noise.shape

TensorShape([1, 66048])

In [10]:
model_best = DiffusionModel(diffusion_steps=diffusion_steps)
model_best.compile(
    optimizer=tfa.optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=weight_decay),
    loss=keras.losses.mean_absolute_error,  # pixelwise mean absolute error is used as loss, mse also ok
)
model_best.load_weights(checkpoint_path)


In [11]:
_, logmel = dataset.mel_fn(speech[None])
data=[]
data.append(speech)
data.append(logmel)
pred_sig,ori_sig = model_best.generate(data,int(0.95*diffusion_steps),noise)
pred_s=pred_sig.numpy()
ori_s=ori_sig.numpy()
#librosa.output.write_wav(os.path.join('/home/lqm/Diffusion-Bison/out/', 'step.wav'),s[0],16000)

2023-04-01 14:38:56.311274: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-01 14:38:56.347110: W tensorflow/stream_executor/cuda/cuda_dnn.cc:342] There was an error before creating cudnn handle: cudaErrorMemoryAllocation : out of memory


signal shape (66048,)
logmel shape (1, 258, 80)
initial_noise shape (1, 66048)
step19...


2023-04-01 14:38:57.125576: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


signal_loss 0.0440080426633358
Mel loss: 3.1987593
step18...
signal_loss 0.04351256042718887
Mel loss: 3.1399357
step17...
signal_loss 0.03655099496245384
Mel loss: 2.895056
step16...
signal_loss 0.03617250546813011
Mel loss: 2.8157704
step15...
signal_loss 0.032315075397491455
Mel loss: 2.5797672
step14...
signal_loss 0.03480013832449913
Mel loss: 2.7183242
step13...
signal_loss 0.03205684944987297
Mel loss: 2.5322673
step12...
signal_loss 0.03129318729043007
Mel loss: 2.4822145
step11...
signal_loss 0.029960177838802338
Mel loss: 2.3162453
step10...
signal_loss 0.02919125370681286
Mel loss: 2.2823093
step9...
signal_loss 0.0292984526604414
Mel loss: 2.3007755
step8...
signal_loss 0.02920069545507431
Mel loss: 2.2474706
step7...
signal_loss 0.02793525718152523
Mel loss: 1.8299419
step6...
signal_loss 0.028817934915423393
Mel loss: 1.9239103
step5...
signal_loss 0.028321731835603714
Mel loss: 1.8756804
step4...
signal_loss 0.03005647286772728
Mel loss: 1.9259465
step3...
signal_loss 0.

In [12]:
librosa.output.write_wav(os.path.join('/home/lqm/Diffusion-Bison/out/', 'best_pred_step.wav'),pred_s,16000)
librosa.output.write_wav(os.path.join('/home/lqm/Diffusion-Bison/out/', 'best_ori_step.wav'),ori_s,16000)

In [13]:
model_final = DiffusionModel(diffusion_steps=diffusion_steps)
model_final.compile(
    optimizer=tfa.optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=weight_decay),
    loss=keras.losses.mean_absolute_error,  # pixelwise mean absolute error is used as loss, mse also ok
)
model_final.load_weights(final_checkpoint_path)


In [14]:
pred_sig,ori_sig = model_final.generate(data,int(0.95*diffusion_steps),noise)
pred_s=pred_sig.numpy()
ori_s=ori_sig.numpy()

signal shape (66048,)
logmel shape (1, 258, 80)
initial_noise shape (1, 66048)
step19...
signal_loss 0.040738631039857864
Mel loss: 3.139355
step18...
signal_loss 0.036955758929252625
Mel loss: 2.9476438
step17...
signal_loss 0.03298822417855263
Mel loss: 2.7424583
step16...
signal_loss 0.036415114998817444
Mel loss: 2.8384395
step15...
signal_loss 0.03396943584084511
Mel loss: 2.7241504
step14...
signal_loss 0.03251845762133598
Mel loss: 2.6080275
step13...
signal_loss 0.03018995374441147
Mel loss: 2.35445
step12...
signal_loss 0.03012385219335556
Mel loss: 2.3001935
step11...
signal_loss 0.03133554011583328
Mel loss: 2.3554063
step10...
signal_loss 0.030574453994631767
Mel loss: 2.2707727
step9...
signal_loss 0.028864117339253426
Mel loss: 2.0836008
step8...
signal_loss 0.02849842980504036
Mel loss: 2.0218234
step7...
signal_loss 0.028412001207470894
Mel loss: 1.9493036
step6...
signal_loss 0.029902905225753784
Mel loss: 1.8463776
step5...
signal_loss 0.03133463114500046
Mel loss: 1.

In [15]:
librosa.output.write_wav(os.path.join('/home/lqm/Diffusion-Bison/out/', 'final_pred_step.wav'),pred_s,16000)
librosa.output.write_wav(os.path.join('/home/lqm/Diffusion-Bison/out/', 'final_ori_step.wav'),ori_s,16000)